# Google Cloud - Retail Search API Quick Start

With Retail Search, retailers can implement Google-quality search that is customizable and built upon Google's understanding of user intent and context.

### Install the libraries used in this tutorial

In [ ]:
pip install --upgrade --user google-cloud-retail

In [ ]:
pip install --upgrade google-auth

In [ ]:
pip install --upgrade --user protobuf

### Global project variables and credentials

In [2]:
import google.auth
#Project Settings
project_id='pod-fr-retail'
project_number='486742359899'
credentials,project = google.auth.default(quota_project_id=project_id)

#Catalog branch represents the version of your catalog
branch= 'projects/{}/locations/global/catalogs/default_catalog/branches/1'.format(project_number)

# Catalog placement represents the search configs you're using on this catalog
placement= 'projects/{}/locations/global/catalogs/default_catalog/placements/default_search'.format(project_id)

#In this quick start, we're going to manage a product id in our catalog
product_id=16684
branch_id=1
placement_id='default_catalog'



## Manage Product Catalog

#### Create a very basic product in your catalog

In [46]:
from google.cloud import retail_v2
# Create a client
client = retail_v2.ProductServiceClient(credentials=credentials)
product=retail_v2.Product()
product.title="dummy product"
product.categories= ["Men > Clothing > Tops & Tees"]
request= retail_v2.CreateProductRequest(
        parent=branch,
        product=product,
        product_id='111222333444',
    )
client.create_product(request=request)

name: "projects/486742359899/locations/global/catalogs/default_catalog/branches/1/products/111222333444"
id: "111222333444"
type_: PRIMARY
primary_product_id: "111222333444"
categories: "Men > Clothing > Tops & Tees"
title: "dummy product"
availability: IN_STOCK

#### Delete this product

In [44]:
from google.cloud import retail_v2
# Create a client
client = retail_v2.ProductServiceClient(credentials=credentials)
request = retail_v2.DeleteProductRequest(
        name=client.product_path(project_number,'global',placement_id,branch_id,111222333444),
    )
client.delete_product(request=request)

#### Get a product from your catalog

In [ ]:

from google.cloud import retail_v2
# Create a client
client = retail_v2.ProductServiceClient(credentials=credentials)
product_name = client.product_path(project_number,'global',placement_id,branch_id,111222333444)
product= client.get_product(name=product_name)
print(product)

#### Update a product - gtin attribute

In [22]:
product.gtin="91234564613"
request = retail_v2.UpdateProductRequest(
    product=product,
)
response = client.update_product(request=request)
product = client.get_product(name=product_name)
print(product.gtin)

91234564613


#### Make sure a product's field will be available in search results

In [18]:
product = client.get_product(name=product_name)
product.retrievable_fields.paths.append('rating')
request = retail_v2.UpdateProductRequest(
    product=product,
)
print(product.retrievable_fields)

# Make the request
#response = client.update_product(request=request)

# Handle the response
#print(response)

paths: "title"
paths: "categories"
paths: "brands"
paths: "attributes.estimated_delivery_day"
paths: "price_info"
paths: "uri"
paths: "images"
paths: "color_info"
paths: "sizes"
paths: "rating"
paths: "rating"
paths: "rating"



### Product Catalog Local Inventories

We are now going to dive into Local inventories. How to set and query these local inventory.
> Documentation https://cloud.google.com/retail/docs/local-inventory-updates
#### Let's start with setting a local inventory for product id 16684 and for store789

In [25]:
from google.cloud import retail_v2

# Create a client
client = retail_v2.ProductServiceClient(credentials=credentials)
product_name = client.product_path(project_number,'global',placement_id,branch_id,product_id)

# Initialize request argument(s)
request = retail_v2.AddLocalInventoriesRequest(
    product=product_name,
    local_inventories= [ 
        retail_v2.LocalInventory( 
            place_id= 'store789'
            , price_info= retail_v2.PriceInfo(
                currency_code= 'USD',
                price = 1.21,
                original_price=1.26
            ))
    ]
)

# Make the request
operation = client.add_local_inventories(request=request, timeout=200)
import time

from IPython.display import display, clear_output
running="Waiting for operation to complete ("
i=0
while operation.running():
    i=i+1
    clear_output(wait=True)
    display(running+str(i)+" seconds)")
    time.sleep(1)
if operation.done():
    clear_output(wait=False)
    display("Done!")

'Done!'

#### Delete a local inventory for product id 16684 and for all stores

In [27]:
# Initialize request argument(s)
request = retail_v2.RemoveLocalInventoriesRequest(
    product=product_name,
    place_ids=[i.place_id for i in product.local_inventories]
)

# Make the request
operation = client.remove_local_inventories(request=request)
import time
from IPython.display import display, clear_output
running="Waiting for operation to complete ("
i=0
while operation.running():
    i=i+1
    clear_output(wait=True)
    display(running+str(i)+" seconds)")
    time.sleep(1)
if operation.done():
    clear_output(wait=False)
    display("Done!")

'Done!'